In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import pandas as pd
df=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [3]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
# from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [5]:
Y=df['Transported'].astype(int)
X=df.loc[:, df.columns != 'Transported']

In [6]:
#Let's perform preprocessing,i.e filter out na values
X['HomePlanet']=X['HomePlanet'].replace(['Europa', 'Earth', 'Mars'], [1, 2, 3])
X['HomePlanet']=X['HomePlanet'].fillna(4)

X['CryoSleep']=X['CryoSleep'].astype(bool).astype(int)
X['CryoSleep']=X['CryoSleep'].fillna(0)

X['Destination']=X['Destination'].replace(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e'], [1, 2, 3])
X['Destination']=X['Destination'].fillna(4)

X['Age']=X['Age'].fillna(0)

X['VIP']=X['VIP'].fillna(False)
X['VIP']=X['VIP'].astype(bool).astype(int)

X['RoomService']=X['RoomService'].fillna(0)

X['FoodCourt']=X['FoodCourt'].fillna(0)

X['ShoppingMall']=X['ShoppingMall'].fillna(0)

X['Spa']=X['Spa'].fillna(0)

X['VRDeck']=X['VRDeck'].fillna(0)

X['VIP']=X['VIP'].astype(bool).astype(int)

X['CryoSleep']=X['CryoSleep'].astype(bool).astype(int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [7]:
X[['passenger_group', 'passenger_number']] = X['PassengerId'].str.split('_', expand=True)
X[['cabin_deck', 'cabin_num','cabin_side']] = X['Cabin'].str.split('/',expand=True)

In [8]:
X['cabin_deck']=X['cabin_deck'].replace(['B', 'F', 'A', 'G', 'E', 'D', 'C', 'T'], [1, 2, 3, 4, 5, 6, 7, 8])
X['cabin_deck']=X['cabin_deck'].fillna(0)
X['cabin_side']=X['cabin_side'].replace(['P','S'], [1, 2])
X['cabin_side']=X['cabin_side'].fillna(0)

X['passenger_group']=X['passenger_group'].fillna(0)
X['passenger_number']=X['passenger_number'].fillna(0)

In [9]:
X.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,passenger_group,passenger_number,cabin_deck,cabin_num,cabin_side
0,0001_01,1.0,0,B/0/P,1.0,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0001,01,1.0,0,1.0
1,0002_01,2.0,0,F/0/S,1.0,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,0002,01,2.0,0,2.0
2,0003_01,1.0,0,A/0/S,1.0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0003,01,3.0,0,2.0
3,0003_02,1.0,0,A/0/S,1.0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0003,02,3.0,0,2.0
4,0004_01,2.0,0,F/1/S,1.0,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,0004,01,2.0,1,2.0


In [10]:
del X['PassengerId']
del X['Cabin']
del X['Name']
del X['cabin_num']


In [11]:
X.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,passenger_group,passenger_number,cabin_deck,cabin_side
0,1.0,0,1.0,39.0,0,0.0,0.0,0.0,0.0,0.0,0001,01,1.0,1.0
1,2.0,0,1.0,24.0,0,109.0,9.0,25.0,549.0,44.0,0002,01,2.0,2.0
2,1.0,0,1.0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0003,01,3.0,2.0
3,1.0,0,1.0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0003,02,3.0,2.0
4,2.0,0,1.0,16.0,0,303.0,70.0,151.0,565.0,2.0,0004,01,2.0,2.0


In [12]:
nan_mask = X.isna()

has_nan = nan_mask.any(axis=1)

X[has_nan]

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,passenger_group,passenger_number,cabin_deck,cabin_side


In [13]:
# decisionTree=tree.DecisionTreeClassifier().fit(X,Y)
rf = RandomForestClassifier(n_estimators=10000, random_state=1).fit(X,Y)

In [14]:
df_test=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
X_test=df_test.loc[:, df_test.columns != 'Transported']

In [15]:
X_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [16]:
X_test['HomePlanet']=X_test['HomePlanet'].replace(['Europa', 'Earth', 'Mars'], [1, 2, 3])
X_test['HomePlanet']=X_test['HomePlanet'].fillna(4)

X_test['CryoSleep']=X_test['CryoSleep'].astype(bool).astype(int)
X_test['CryoSleep']=X_test['CryoSleep'].fillna(0)

X_test['Destination']=X_test['Destination'].replace(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e'], [1, 2, 3])
X_test['Destination']=X_test['Destination'].fillna(4)

X_test['Age']=X_test['Age'].fillna(0)

X_test['VIP']=X_test['VIP'].fillna(False)
X_test['VIP']=X_test['VIP'].astype(bool).astype(int)

X_test['RoomService']=X_test['RoomService'].fillna(0)

X_test['FoodCourt']=X_test['FoodCourt'].fillna(0)

X_test['ShoppingMall']=X_test['ShoppingMall'].fillna(0)

X_test['Spa']=X_test['Spa'].fillna(0)

X_test['VRDeck']=X_test['VRDeck'].fillna(0)

X_test[['passenger_group', 'passenger_number']] = X_test['PassengerId'].str.split('_', expand=True)

X_test[['cabin_deck', 'cabin_num','cabin_side']] = X_test['Cabin'].str.split('/',expand=True)

In [17]:
X_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,passenger_group,passenger_number,cabin_deck,cabin_num,cabin_side
0,0013_01,2.0,1,G/3/S,1.0,27.0,0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,0013,01,G,3,S
1,0018_01,2.0,0,F/4/S,1.0,19.0,0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0018,01,F,4,S
2,0019_01,1.0,1,C/0/S,3.0,31.0,0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0019,01,C,0,S
3,0021_01,1.0,0,C/1/S,1.0,38.0,0,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0021,01,C,1,S
4,0023_01,2.0,0,F/5/S,1.0,20.0,0,10.0,0.0,635.0,0.0,0.0,Brence Harperez,0023,01,F,5,S


In [18]:
X_test['cabin_deck']=X_test['cabin_deck'].replace(['B', 'F', 'A', 'G', 'E', 'D', 'C', 'T'], [1, 2, 3, 4, 5, 6, 7, 8])
X_test['cabin_deck']=X_test['cabin_deck'].fillna(0)
X_test['cabin_side']=X_test['cabin_side'].replace(['P','S'], [1, 2])
X_test['cabin_side']=X_test['cabin_side'].fillna(0)

X_test['passenger_group']=X_test['passenger_group'].fillna(0)
X_test['passenger_number']=X_test['passenger_number'].fillna(0)
# X_test['Expenses']=X_test['FoodCourt']+X_test['ShoppingMall']+X_test['Spa']+X_test['VRDeck']+X_test['RoomService']

In [19]:
del X_test['PassengerId']
# del X_test['passenger_number']
del X_test['Cabin']
del X_test['Name']
del X_test['cabin_num']

In [20]:
X_test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,passenger_group,passenger_number,cabin_deck,cabin_side
0,2.0,1,1.0,27.0,0,0.0,0.0,0.0,0.0,0.0,0013,01,4.0,2.0
1,2.0,0,1.0,19.0,0,0.0,9.0,0.0,2823.0,0.0,0018,01,2.0,2.0
2,1.0,1,3.0,31.0,0,0.0,0.0,0.0,0.0,0.0,0019,01,7.0,2.0
3,1.0,0,1.0,38.0,0,0.0,6652.0,0.0,181.0,585.0,0021,01,7.0,2.0
4,2.0,0,1.0,20.0,0,10.0,0.0,635.0,0.0,0.0,0023,01,2.0,2.0


In [21]:
y_pred = rf.predict(X_test)

In [22]:
result = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Transported': y_pred})

In [23]:
result.head()

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,1
4,0023_01,1


In [24]:
result['Transported']=result['Transported'].replace([1,0], [True,False])

In [25]:
result.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [26]:
result.to_csv('/kaggle/working/submissions.csv', index=False)

In [27]:
len(result)

4277